In [1]:
import numpy as np
import math
import os.path
import csv
import glob
import tensorflow as tf
import h5py as h5py
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model

Using TensorFlow backend.


In [2]:
import imageio
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

In [3]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [4]:
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar

In [5]:
def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

In [6]:
def mouth_over_eye(eye):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

In [7]:
def getFrame(sec):
    start = 180000
    vidcap.set(cv2.CAP_PROP_POS_MSEC, start + sec*1000)
    hasFrames,image = vidcap.read()
    return hasFrames, image

In [8]:
# main program
from scipy.spatial import distance
import cv2
data = []
labels = []
for j in [60]:
  for i in [10]:
    vidcap = cv2.VideoCapture('10_1.mp4')
    sec = 0
    frameRate = 1
    success, image  = getFrame(sec)
    count = 0
    while success and count < 240: 
          landmarks = extract_face_landmarks(image)
          if sum(sum(landmarks)) != 0:
              count += 1
              data.append(landmarks)
              labels.append([i])
              sec = sec + frameRate
              sec = round(sec, 2)
              success, image = getFrame(sec)
              print(count)
          else:  
              sec = sec + frameRate
              sec = round(sec, 2)
              success, image = getFrame(sec)
              print("not detected")
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240


In [9]:
import numpy as np
data = np.array(data)
labels = np.array(labels)

In [10]:
features = []
for d in data:
  eye = d[36:68]
  ear = eye_aspect_ratio(eye)
  mar = mouth_aspect_ratio(eye)
  cir = circularity(eye)
  mouth_eye = mouth_over_eye(eye)
  features.append([ear, mar, cir, mouth_eye])

In [11]:
features = np.array(features)
features.shape

(240, 4)

In [12]:
data.shape

(240, 68, 2)

In [13]:
labels.shape

(240, 1)

In [14]:
# save files 
np.save(open('Sample_Data.npy', 'wb'),data)
np.save(open('Sample_Features.npy', 'wb'),features)
np.save(open('Sample_Labels.npy', 'wb'),labels)
np.savetxt("Sample_Features.csv", features, delimiter = ",")
np.savetxt("Sample_Labels.csv", labels, delimiter = ",")